## 사전 작업

In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용
import matplotlib.font_manager as fm
# 설치된 폰트 중 'NanumGothic' 찾기
for font in fm.fontManager.ttflist:
    if 'NanumGothic' in font.name:
        plt.rcParams['font.family'] = font.name
        break

plt.rcParams['axes.unicode_minus'] = False

### 데이터 로드

In [2]:
df_symptoms = pd.read_csv("./csv/djkim_helpline_Symptom.csv")
df_symptoms[:2]

,_id,disease_korean_title,related_diseases,diseases_symptoms,cause_diseases
0,65dc4517da45daab46acd344,5번 염색체 장완 14.3 부분의 미세결손 증후군,5번 염색체 장완의 14.3 부분의 결손 증후군\n(5q14.3 deletion s...,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,\n근긴...",5번 염색체 장완의 14.3 부분의 미세결손
1,65dc451dda45daab46acd346,7번 염색체 장완 36 부분의 미세결손 증후군,미세결실 증후군\n(microdeletion syndrome),"발달 지연, 안면 기형",7번 염색체 장완 유전자 말단 부위의 미세 결실


### 질환의 증상에 대한 줄바꿈 기호 전처리

In [3]:
import re
df_symptoms['diseases_symptoms'] = df_symptoms['diseases_symptoms'].astype(str) 
patterns = '(|\n|\\\\n|)'
series = []
for contents in df_symptoms['diseases_symptoms']:
    text_regex = re.sub(pattern=patterns,repl='',string=contents)
    series.append(text_regex)
series[:2]

['중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,근긴장저하, 운동발달지연, 자폐증상, 발가락 합지증, 눈맞춤 늦음.',
 '발달 지연, 안면 기형']

In [4]:
df_symptoms['diseases_symptoms'] = series
df_symptoms['diseases_symptoms'][:2]

0    중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,근긴장저...
1                                         발달 지연, 안면 기형
Name: diseases_symptoms, dtype: object

### 전문 용어의 특수성을 감안하여 모델을 분리하여 토크나이징 착수

In [5]:
df1 = df_symptoms[['disease_korean_title', 'diseases_symptoms']]
df2 = df1.copy()

### 2번 데이터프레임에 대해 전처리

In [6]:
from konlpy.tag import Okt
okt = Okt()

In [7]:
# 조사, 어미, 구두점 등 제거
def okt_clean(text):
    clean_text = []
    for word in okt.pos(text, stem=True):
        if word[1] not in ['Josa', 'Eomi', 'Punctuation']:
            clean_text.append(word[0])

    return " ".join(clean_text)

In [8]:
df2['okt_korean_title']= df2['disease_korean_title'].apply(okt_clean)
df2['okt_korean_title'][:2]

0    5 번 염색체 장완 14.3 부분 미세 결손 증후군
1      7 번 염색체 장완 36 부분 미세 결손 증후군
Name: okt_korean_title, dtype: object

In [9]:
df2['okt_symptoms']= df2['diseases_symptoms'].apply(okt_clean)
df2['okt_symptoms'][:2]

0    중증 정신지체 언어 발달 지연 상동 행동 뇌전증 뇌 기형 특징 적 생김새 근 긴장 ...
1                                          발달 지연 안면 기형
Name: okt_symptoms, dtype: object

In [10]:
df2[:2]

,disease_korean_title,diseases_symptoms,okt_korean_title,okt_symptoms
0,5번 염색체 장완 14.3 부분의 미세결손 증후군,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,근긴장저...",5 번 염색체 장완 14.3 부분 미세 결손 증후군,중증 정신지체 언어 발달 지연 상동 행동 뇌전증 뇌 기형 특징 적 생김새 근 긴장 ...
1,7번 염색체 장완 36 부분의 미세결손 증후군,"발달 지연, 안면 기형",7 번 염색체 장완 36 부분 미세 결손 증후군,발달 지연 안면 기형


In [11]:
df3 = df2[['okt_korean_title','okt_symptoms']]
df3[:2]

,okt_korean_title,okt_symptoms
0,5 번 염색체 장완 14.3 부분 미세 결손 증후군,중증 정신지체 언어 발달 지연 상동 행동 뇌전증 뇌 기형 특징 적 생김새 근 긴장 ...
1,7 번 염색체 장완 36 부분 미세 결손 증후군,발달 지연 안면 기형


### Tokenizing

In [12]:
# tokeninzing
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer1 = TfidfVectorizer(token_pattern=r"(?u)\b\w\w+\b|[\!\?\.,;:@#\$%\^&\*\(\)-_=\+`\~\[\]\{\}\'\"]", ngram_range=(1,2))
tfidfVectorizer2 = TfidfVectorizer(tokenizer=None, ngram_range=(1,2))

In [13]:
vector_symptoms_1 = tfidfVectorizer1.fit(df1['diseases_symptoms'])

In [14]:
vector_symptoms_2 = tfidfVectorizer2.fit(df3['okt_symptoms'])

### 모델화

In [15]:
transform_symptoms_1 = tfidfVectorizer1.fit_transform(df1['diseases_symptoms'])
transform_symptoms_2 = tfidfVectorizer2.fit_transform(df3['okt_symptoms'])

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(transform_symptoms_1, df1['disease_korean_title'], test_size=0.2, random_state=42)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(transform_symptoms_2, df3['okt_korean_title'], test_size=0.2, random_state=42)

model_1 = LogisticRegression()
model_2 = LogisticRegression()

model_1.fit(X_train_1, y_train_1)
model_2.fit(X_train_2, y_train_2)


LogisticRegression()

In [17]:
import pickle

In [18]:
with open('model_1.pkl', 'wb') as f:
    pickle.dump(model_1, f)

with open('model_2.pkl', 'wb') as f:
    pickle.dump(model_2, f)

In [19]:
with open('model_1.pkl', 'rb') as f:
    loaded_model_1 = pickle.load(f)

with open('model_2.pkl', 'rb') as f:
    loaded_model_2 = pickle.load(f)

### 평가

In [75]:
from sklearn.metrics import accuracy_score
y_pred_1 = model_1.predict(X_test_1)
y_pred_2 = model_2.predict(X_test_2)
accuracy_1 = accuracy_score(y_test_1, y_pred_1)
print(f'Model 1 Accuracy: {accuracy_1}')

accuracy_2 = accuracy_score(y_test_2, y_pred_2)
print(f'Model 2 Accuracy: {accuracy_2}')

Model 1 Accuracy: 0.12554112554112554
Model 2 Accuracy: 0.12554112554112554


### 재수립

In [24]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤 포레스트 모델을 생성합니다.
model_1_rf = RandomForestClassifier(random_state=42)
model_2_rf = RandomForestClassifier(random_state=42)

# 각각의 데이터에 대해 모델을 학습시킵니다.
model_1_rf.fit(X_train_1, y_train_1)
model_2_rf.fit(X_train_2, y_train_2)

RandomForestClassifier(random_state=42)

In [76]:
y_pred_1_rf = model_1_rf.predict(X_test_1)
y_pred_2_rf = model_2_rf.predict(X_test_2)

accuracy_1_rf = accuracy_score(y_test_1, y_pred_1_rf)
print(f'Random Forest Model 1 Accuracy: {accuracy_1_rf}')

accuracy_2_rf = accuracy_score(y_test_2, y_pred_2_rf)
print(f'Random Forest Model 2 Accuracy: {accuracy_2_rf}')

Random Forest Model 1 Accuracy: 0.1774891774891775
Random Forest Model 2 Accuracy: 0.1774891774891775


In [27]:
model_1_rf = RandomForestClassifier(n_estimators=200, max_depth=None, min_samples_split=2, random_state=42)
model_2_rf = RandomForestClassifier(n_estimators=200, max_depth=None, min_samples_split=2, random_state=42)

model_1_rf.fit(X_train_1, y_train_1)
model_2_rf.fit(X_train_2, y_train_2)

RandomForestClassifier(n_estimators=200, random_state=42)

In [77]:
y_pred_1_rf = model_1_rf.predict(X_test_1)
y_pred_2_rf = model_2_rf.predict(X_test_2)

accuracy_1_rf = accuracy_score(y_test_1, y_pred_1_rf)
print(f'Random Forest Model 1 Accuracy: {accuracy_1_rf}')

accuracy_2_rf = accuracy_score(y_test_2, y_pred_2_rf)
print(f'Random Forest Model 2 Accuracy: {accuracy_2_rf}')

Random Forest Model 1 Accuracy: 0.1774891774891775
Random Forest Model 2 Accuracy: 0.1774891774891775


In [29]:
from sklearn.metrics import classification_report

In [48]:
test_symptoms = df1['diseases_symptoms']
test_symptoms

0       중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,근긴장저...
1                                            발달 지연, 안면 기형
2       발달 지연, 지적 장애, 뇌전증, 두개안면기형, 중추신경계 기형, 근골격계 기형, ...
3                       언어발달 지연, 학습장애, 자폐스펙트럼장애, 근육긴장저하 등
4         시력상실 (blindness), 청력상실(deafness), 발달지연 등 신경계 이상
                              ...                        
1150                                 빈혈, 창백함, 피로, 가쁜 호흡 등
1151                                            만성 용혈성 빈혈
1152    빈혈, 중등도의 만성 용혈, 황달, 비장비대, 빠른 심박수, 빠른 호흡, 창백함, ...
1153                                 만성 용혈성 빈혈, 신경기능학적 이상
1154                                            만성 용혈성 빈혈
Name: diseases_symptoms, Length: 1155, dtype: object

In [34]:
# def tokenize_symptoms(symptoms):
#     return okt.pos(symptoms)

# # 증상을 모델이 인식할 수 있는 형태로 변환하는 함수를 정의합니다.
# def transform_input(symptoms, vectorizer):
#     # 증상을 토크나이징합니다.
#     tokenized_symptoms = symptoms.apply(tokenize_symptoms)
#     # 토크나이징된 증상을 문자열로 변환합니다.
#     tokenized_symptoms_str = tokenized_symptoms.apply(' '.join)
#     # 문자열로 변환된 증상을 벡터화하여 반환합니다.
#     return vectorizer.transform(tokenized_symptoms_str)

In [49]:
Symptoms_transformed = tfidfVectorizer1.transform(test_symptoms)

In [50]:
predictions = model_1_rf.predict(Symptoms_transformed)
predictions

array(['5번 염색체 장완 14.3 부분의 미세결손 증후군', '7번 염색체 장완 36 부분의 미세결손 증후군',
       '13번 염색체 장완의 부분 결손', ..., '피루브산염카이네이스결핍빈혈', '삼탄당인산염이성화효소결핍빈혈',
       '헥소카이네이스결핍빈혈'], dtype=object)

In [51]:
df1

,disease_korean_title,diseases_symptoms
0,5번 염색체 장완 14.3 부분의 미세결손 증후군,"중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,근긴장저..."
1,7번 염색체 장완 36 부분의 미세결손 증후군,"발달 지연, 안면 기형"
2,13번 염색체 장완의 부분 결손,"발달 지연, 지적 장애, 뇌전증, 두개안면기형, 중추신경계 기형, 근골격계 기형, ..."
3,17번 염색체 장완 21.31 부분의 미세중복 증후군,"언어발달 지연, 학습장애, 자폐스펙트럼장애, 근육긴장저하 등"
4,노리에병,"시력상실 (blindness), 청력상실(deafness), 발달지연 등 신경계 이상"
...,...,...
1150,베타지중해빈혈,"빈혈, 창백함, 피로, 가쁜 호흡 등"
1151,해당효소의 장애에 의한 빈혈,만성 용혈성 빈혈
1152,피루브산염카이네이스결핍빈혈,"빈혈, 중등도의 만성 용혈, 황달, 비장비대, 빠른 심박수, 빠른 호흡, 창백함, ..."
1153,삼탄당인산염이성화효소결핍빈혈,"만성 용혈성 빈혈, 신경기능학적 이상"


In [52]:
# F1 Score 적용
from sklearn.metrics import classification_report

In [72]:
def tokenize_symptoms(symptoms):
    return okt.pos(symptoms)
def transform_input(symptoms, vectorizer):
    tokenized_symptoms = [word for word, pos in tokenize_symptoms(symptoms)]
    tokenized_symptoms_str = ' '.join(tokenized_symptoms)
    return tfidfVectorizer1.transform([tokenized_symptoms_str])
symptoms_input = input("Enter symptoms: ")
symptoms_transformed = transform_input(symptoms_input, tfidfVectorizer1)
prediction = model_1_rf.predict(symptoms_transformed)
print(f'The prediction is: {prediction}')

Enter symptoms:  시력상실


The prediction is: ['선천성 경상 운동 장애']


In [71]:
# 증상을 입력받습니다.
symptoms_input = input("Enter symptoms: ")
symptoms_transformed = transform_input(symptoms_input, tfidfVectorizer1)
prediction = model_1_rf.predict(symptoms_transformed)
print(f'The prediction is: {prediction}')

Enter symptoms:  빈혈


The prediction is: ['쿠울리빈혈']


In [70]:
symptoms_input = input("Enter symptoms: ")
symptoms_transformed = transform_input(symptoms_input, tfidfVectorizer1)
prediction = model_1_rf.predict(symptoms_transformed)
print(f'The prediction is: {prediction}')

Enter symptoms:  황달


The prediction is: ['배꼽내장탈장']


In [74]:
df1['diseases_symptoms']

0       중증 정신지체, 언어발달지연, 상동행동, 뇌전증, 뇌기형, 특징적인 생김새,근긴장저...
1                                            발달 지연, 안면 기형
2       발달 지연, 지적 장애, 뇌전증, 두개안면기형, 중추신경계 기형, 근골격계 기형, ...
3                       언어발달 지연, 학습장애, 자폐스펙트럼장애, 근육긴장저하 등
4         시력상실 (blindness), 청력상실(deafness), 발달지연 등 신경계 이상
                              ...                        
1150                                 빈혈, 창백함, 피로, 가쁜 호흡 등
1151                                            만성 용혈성 빈혈
1152    빈혈, 중등도의 만성 용혈, 황달, 비장비대, 빠른 심박수, 빠른 호흡, 창백함, ...
1153                                 만성 용혈성 빈혈, 신경기능학적 이상
1154                                            만성 용혈성 빈혈
Name: diseases_symptoms, Length: 1155, dtype: object